In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge
import torch
import pandas as pd

In [ ]:
model_name = 'bigscience/mt0-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [30]:
def get_response_and_scores(prompt, reference):
    # Encode & Decode
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=50, truncation=True)
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    input_ids = tokenizer.encode(response, return_tensors='pt')

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        
    loss = outputs.loss
    perplexity = torch.exp(loss)

    rouge = Rouge()
    scores = rouge.get_scores(response, reference)

    df = pd.DataFrame({
        'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
        'Recall': [scores[0]['rouge-1']['r'], scores[0]['rouge-2']['r'], scores[0]['rouge-l']['r']],
        'Precision': [scores[0]['rouge-1']['p'], scores[0]['rouge-2']['p'], scores[0]['rouge-l']['p']],
        'F1 Score': [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']],
    })

    print(f'ROUGE scores\n{df}')

    return response, perplexity.item(), scores

In [31]:
new_prompt = 'Imagine you are an expert on AI. Explain the impact of AI.'
reference_text = 'AI has a significant impact on various fields, including healthcare, transportation, and finance. It can automate tasks, improve decision-making, and provide new insights from data.'

response, perplexity, rouge_scores = get_response_and_scores(new_prompt, reference_text)
print('\nResponse:', response)
print('Perplexity:', perplexity)


ROUGE scores
    Metric    Recall  Precision  F1 Score
0  ROUGE-1  0.125000   0.230769  0.162162
1  ROUGE-2  0.000000   0.000000  0.000000
2  ROUGE-L  0.083333   0.153846  0.108108

Response: <pad> Scientists are increasingly concerned about the impact of AI on humans.</s>
Perplexity: 4.322360992431641
